In [103]:
import socket 
import sys
import json
import requests
import time
from scapy.all import *

In [105]:
DEBUG_MOD = True
MAIN_TARGET = None

In [107]:
def run_dublin_traceroute():
    # 執行 Dublin-traceroute
    print(f'正在對 {MAIN_TARGET} 執行 Dublin-traceroute...', end='')
    os.system(f'dublin-traceroute -n 5 {MAIN_TARGET} > /dev/null')
    # 讀取路徑追蹤資料
    with open('trace.json', 'r') as f:
        trace_data = json.load(f)
        print('完成')
        return trace_data

In [108]:
def snmp_helper(trace_data):
    # 儲存 TTL 與 IP 位址的對應關係
    ttl_history = {}
    # 針對每一條路徑追蹤資料進行處理
    for flow_id in trace_data['flows']:
        flow = trace_data['flows'][flow_id]
        if DEBUG_MOD:
            print(f'--{flow_id}--')
        
        # 針對每一個跳躍回應進行處理
        for hop_response_id in range(0,len(flow)):
            ttl = hop_response_id+1
            receive_ip = None if not flow[hop_response_id]['received'] else flow[hop_response_id]['received']['ip']['src']
            # 如果沒有回應，則嘗試從 TTL 對應的 IP 位址紀錄中取得結果
            # 目前先不開紀錄，確保封包探測的準確率提高
            if receive_ip==None:
                # if ttl in ttl_history:
                #     trace_data['flows'][flow_id][hop_response_id]=ttl_history[ttl]
                # else:
                #     trace_data['flows'][flow_id][hop_response_id]=packet_track(ttl, MAIN_TARGET)
                trace_data['flows'][flow_id][hop_response_id]=None
            # 如果有回應，則直接將 IP 位址紀錄到路徑追蹤資料中
            else:
                temp = trace_data['flows'][flow_id][hop_response_id]['received']['ip']['src']
                trace_data['flows'][flow_id][hop_response_id] = temp
                if DEBUG_MOD:
                    print(f'TTL {ttl} 為 Dublin-traceroute 找到的結果為 {temp}')
    if DEBUG_MOD:
        print(trace_data)
    return trace_data


In [109]:
def packet_track(ttl, target_ip_address, test_count=3, wait_time=2):
    # 回傳可能的IP集合
    res = set()
    # 建立 ICMP 封包
    trace_type = [ICMP(), TCP(flags='S'), UDP()]
    for t in trace_type:
        packet = IP(dst=target_ip_address, ttl=ttl) / t
        for i in range(test_count):
            # 傳送封包並接收回傳的封包
            response_packet = sr1(packet, verbose=False, timeout=wait_time)
            # 檢查是否有回傳的封包
            if response_packet is not None:
                # 印出回傳的封包資訊
                res.add(response_packet.getlayer(IP).src)
    if DEBUG_MOD:
        print(f'TTL {ttl} 為封包探測找到的結果 {list(res)} (只是猜測)')

    return list(res) if len(res)!=0 else None

In [110]:
def main(ip):
    global MAIN_TARGET
    MAIN_TARGET = ip
    # 從命令列參數檢查目標 IP 位址
    search = requests.get(f'http://144.24.84.156/path?ip={MAIN_TARGET}').json()
    if search != None and time.time()-float(search['time'])<300:
        with open('trace_optimized.json', 'w') as f:
            f.write(search['value'])
        print("完成！(根據紀錄資料)")
    else:
        # 執行 Dublin Traceroute，取得路徑追蹤資料
        trace_data = run_dublin_traceroute()
        # 透過 SNMP 協助最佳化路徑追蹤資料
        trace_data_after_snmp_asistance = snmp_helper(trace_data)
        # 將最佳化後的路徑追蹤資料寫入 JSON 檔案
        with open('trace_optimized.json', 'w') as f:
            f.write(json.dumps(trace_data_after_snmp_asistance))
        requests.post(f'http://144.24.84.156/path',data={'ip':MAIN_TARGET,'time':str(time.time()),'value':str(json.dumps(trace_data_after_snmp_asistance))})
        # 印出完成訊息
        print("完成！")

In [112]:
if __name__ == '__main__':
    main()

完成！(根據紀錄資料)
